In [ ]:
%matplotlib inline
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import json

In [ ]:
with open('data/parameterised_random.json') as infile:
    data = json.load(infile)

In [ ]:
data[0]

In [ ]:
df = pd.DataFrame([
    {
        'param_rho': instance['params']['lhs_params']['density'],
        'param_pv': instance['params']['lhs_params']['pv'],
        'param_pc': instance['params']['lhs_params']['pc'],
        'basis_split': instance['params']['beta_params']['basis_split'],
        'beta_param': pd.np.log(instance['params']['alpha_params']['beta_param']),
        'frac_violations': instance['params']['alpha_params']['frac_violations'],
        'nonzeros': instance['nonzeros'],
        'var_degree_max': instance['var_degree_max'],
        'cons_degree_max': instance['cons_degree_max'],
        'var_degree_min': instance['var_degree_min'],
        'cons_degree_min': instance['cons_degree_min'],
        'fractional_primal': instance['fractional_primal'],
        'binding_constraints': instance['binding_constraints'],
        'total_fractionality': instance['total_fractionality'],
    }
    for instance in data
])
sns.pairplot(df,
             x_vars=['param_rho', 'param_pv', 'param_pc', 'basis_split', 'beta_param', 'frac_violations'],
             y_vars=[
                'nonzeros', 'var_degree_min', 'var_degree_max', 'cons_degree_min', 'cons_degree_max',
                 'fractional_primal', 'binding_constraints', 'total_fractionality'])

# Controlling min/max/mean

The aim here is to show that rho controls number of nonzeros, pv/pc control variable and constraint skew *independently of density*.
We also want parameters to be independent of size, hence the use of fractional values.

In [ ]:
plt.scatter(x=df.param_rho, y=df.nonzeros)
plt.xlabel('Density Parameter')
plt.ylabel('Number of Nonzeros')

In [ ]:
plt.scatter(x=df.param_rho, y=df.param_pc, c=df.nonzeros)

In [ ]:
plt.scatter(x=df.param_rho, y=df.param_pc, c=df.nonzeros)
cbar = plt.colorbar()
cbar.set_label('Number of Nonzeros')
plt.xlabel('Density Parameter')
plt.ylabel('PC Parameter')

In [ ]:
plt.scatter(x=df.param_rho, y=df.param_pc, c=df.cons_degree_max - df.cons_degree_min)
cbar = plt.colorbar()
cbar.set_label('Max - Min Constraint Degree')
plt.xlabel('Density Parameter')
plt.ylabel('PC Parameter')

In [ ]:
plt.scatter(x=df.param_rho, y=df.param_pc, c=df.cons_degree_min)
cbar = plt.colorbar()
cbar.set_label('Min Constraint Degree')
plt.xlabel('Density Parameter')
plt.ylabel('PC Parameter')

In [ ]:
plt.scatter(x=df.param_rho, y=df.param_pv, c=df.var_degree_max)

In [ ]:
plt.scatter(x=df.param_rho, y=df.param_pv, c=df.var_degree_min)

In [ ]:
df.head()

In [ ]:
plt.scatter(x=df.basis_split, y=df.frac_violations, c=df.fractional_primal)

In [ ]:
plt.scatter(x=df.basis_split * df.frac_violations, y=df.beta_param, c=df.total_fractionality)

In [ ]:
import statsmodels.api as sm

In [ ]:
model = sm.OLS(df['nonzeros'], df[['param_rho', 'param_pv', 'param_pc']])
results = model.fit()
results.summary()

In [ ]:
model = sm.OLS(df['cons_degree_min'], df[['param_rho', 'param_pv', 'param_pc']])
results = model.fit()
results.summary()

In [ ]:
model = sm.OLS(df['cons_degree_max'], df[['param_rho', 'param_pv', 'param_pc']])
results = model.fit()
results.summary()